In [1]:
DATA_NAME = 'syntheticMRI2D-sagittal' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_diagonal_wavelet_sagittal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,269725.65,0.12,1.16,-1.85,0.01,-0.08,-0.02
4,0.12,175434.87,-2.10,-1.47,0.91,0.07,0.02
5,1.16,-2.10,35102.19,0.16,0.31,0.18,-0.02
6,-1.85,-1.47,0.16,13482.74,0.01,-0.01,-0.00
7,0.01,0.91,0.31,0.01,2081.60,0.01,0.00
8,-0.08,0.07,0.18,-0.01,0.01,127.36,0.00
9,-0.02,0.02,-0.02,-0.00,0.00,0.00,5.92


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,0.00000,0.00001,-0.00003,0.00000,-0.00001,-0.00001
4,0.00000,1.00000,-0.00003,-0.00003,0.00005,0.00001,0.00002
5,0.00001,-0.00003,1.00000,0.00001,0.00004,0.00008,-0.00005
6,-0.00003,-0.00003,0.00001,1.00000,0.00000,-0.00000,-0.00002
7,0.00000,0.00005,0.00004,0.00000,1.00000,0.00002,0.00004
8,-0.00001,0.00001,0.00008,-0.00000,0.00002,1.00000,0.00001
9,-0.00001,0.00002,-0.00005,-0.00002,0.00004,0.00001,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

4.960123888803757

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.06020421e+10 4.20176845e+09 1.65720288e+08 2.25359661e+07
 4.80409121e+05 1.35324233e+03 5.21335379e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.983806,-0.178110,-0.018752,-0.007092,-0.001092,-0.000067,-0.000003
1,0.176971,0.983008,-0.045678,-0.016814,-0.002571,-0.000157,-0.000007
2,0.025272,0.039626,0.992542,-0.111584,-0.014131,-0.000856,-0.000041
3,0.012736,0.019631,0.109658,0.992193,-0.054500,-0.003228,-0.000151
4,0.002579,0.003960,0.019849,0.052410,0.997938,-0.030895,-0.001427
5,0.000236,0.000361,0.001804,0.004711,0.030559,0.998437,-0.046513
6,0.000022,0.000034,0.000169,0.000439,0.002840,0.046446,0.998917


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.01619428872303663,
 0.01699165636823352,
 0.007457957376072244,
 0.007807042393323638,
 0.0020619482300494596,
 0.0015626841403060254,
 0.0010833466294811744]